# Fuzzy Comparison between human-made and AI-made data

In [2]:
import pandas as pd

# Load XLS file with the date column explicitly set to string type
xls_path = 'All_Concerts_1908_filtered.xlsx'

# Load the Excel file, skipping the specified number of rows
xls_df = pd.read_excel(xls_path)

# Display the first few rows of the loaded DataFrame to verify
print(xls_df.head(10))

# Display the DataFrame to ensure dates are read as strings. Basically if you see dates here and not NaT, we're good.
#start_row_xls = 2090
#num_rows_to_display = 3  # Adjust as needed
#print("Excel DataFrame from row 2090:") # reason: the beginning of the data doesn't have dates in the date column.
#print(xls_df.iloc[start_row_xls:start_row_xls + num_rows_to_display])

   konsert_ID konsert_datum                         konsert_namn  \
0        2528    1908-01-02    Brüssel-kvartetten Beethovenafton   
1        2509    1908-01-05  1:sta säsångskonsert: Nordisk afton   
2        2521    1908-01-05                       Afskedskonsert   
3        2516    1908-01-05                         Folk-konsert   
4        2515    1908-01-05                       Populärkonsert   
5        2510    1908-01-06        2:a populära Symfoni-matinéen   
6        2511    1908-01-06                     Jul-fest med Bal   
7        2517    1908-01-06                        Populär soaré   
8        2519    1908-01-06                        Populär soaré   
9        2512    1908-01-07  2:a kammarmusikkonsert: Brahmsafton   

                                     lokal_namn  \
0         Kungliga musikaliska akademiens lokal   
1                    Hotel Continentals festsal   
2                               Immanuelskyrkan   
3  Arbetare-institutet, Klara Norra Kyrkogata 8

In [3]:

# Load JSON file
json_path = 'Datasets/18.05.24_oldtimey_Touringbot_SvD_1908-01-01-1908-02-29_search_konsert_gpt3.5_turbo_batch_cpkGJ53jIRLdMmZGbhlkL7cT_output.json'
json_df = pd.read_json(json_path)

#Did this load right? If there's dates and whatever in here, we're good.
print(json_df.head(3))

  konsert_datum                     konsert_namn  \
0      15.02.08                 Symfonikonserten   
1      14.02.08               Populär Sång-Afton   
2      21.02.08  Vivien Cliartrcs afskedskonsert   

                                  lokal_namn  konsert_biljettpris  \
0                           Kungliga Teatern              UNKNOWN   
1       F.U.M:s hörsal, Birger Jarlsgatan 35      1kr 50 öre, 1kr   
2  Kungliga musikaliska akademiens stora sal  2:50kr, 2kr, 1:50kr   

  SummaförAnnons/program SummaförRecension SummaförAntalförkonsert_verk_ID  \
0                      0                 0                               4   
1                      1                 0                               0   
2                      1                 0                               7   

         Arrangör konserttyp_namn     Månad    År       Dag Månadnr  \
0   Järnefelt, A.  Symfonikonsert  FEBRUARY  1908  Saturday       2   
1  Lundberg, Anna     Sångkonsert  FEBRUARY  1908    Friday

# Now we normalise the text so that they can be compared

## Removing Columns that are irrelevant

In [4]:
# Identify relevant columns (adjust based on your specific dataset)
relevant_columns = ['konsert_datum','konsert_namn','lokal_namn','konsert_biljettpris','Arrangör','konserttyp_namn','Producer']

# Remove irrelevant columns from both DataFrames
xls_df = xls_df[relevant_columns]
json_df = json_df[relevant_columns]


In [5]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
from datetime import datetime

# Function to normalize text
def normalize(text):
    if pd.isna(text) or text.strip().lower() == "unknown":  # Handle NaN values and "UNKNOWN"
        return ""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[-,]', ' ', text)  # Replace hyphens and commas with spaces
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing spaces
    return text

# Function to normalize dates
def normalize_date(date):
    try:
        # Try to parse the date from different formats
        if isinstance(date, str):
            if re.match(r'\d{2}\.\d{2}\.\d{2}', date):  # Format: DD.MM.YY
                date_obj = datetime.strptime(date, '%d.%m.%y')
                # Adjust for century
                if date_obj.year >= 2000:
                    date_obj = date_obj.replace(year=date_obj.year - 100)
            elif re.match(r'\d{4}\.\d{2}\.\d{2}', date):  # Format: YYYY.MM.DD
                date_obj = datetime.strptime(date, '%Y.%m.%d')
            elif re.match(r'\d{2}\.\d{2}\.\d{4}', date):  # Format: DD.MM.YYYY
                date_obj = datetime.strptime(date, '%d.%m.%Y')
            else:
                return ""
        elif isinstance(date, datetime):
            # If already a datetime object, just format it
            date_obj = date
        else:
            return ""
        return date_obj.strftime('%Y-%m-%d')  # Convert to "YYYY-MM-DD"
    except ValueError:
        return ""

# Replace 'UNKNOWN' with an empty string in the entire DataFrame
xls_df = xls_df.replace('UNKNOWN', '')
json_df = json_df.replace('UNKNOWN', '')

# Normalize the date columns in both dataframes
xls_df['normalized_konsert_datum'] = xls_df['konsert_datum'].apply(normalize_date)
json_df['normalized_konsert_datum'] = json_df['konsert_datum'].apply(normalize_date)

# Normalize all text columns in both dataframes
for col in xls_df.columns:
    if xls_df[col].dtype == 'object':  # Apply only to text columns
        xls_df[col] = xls_df[col].apply(normalize)

for col in json_df.columns:
    if json_df[col].dtype == 'object':  # Apply only to text columns
        json_df[col] = json_df[col].apply(normalize)

# Columns to compare (excluding date)
columns_to_compare = ['konsert_namn', 'lokal_namn', 'konsert_biljettpris', 'Arrangör', 'konserttyp_namn', 'Producer']

# Function to compare rows with the same date
def compare_rows_on_date(date, df1, df2, columns):
    # Get rows with the same date from both dataframes
    rows1 = df1[df1['normalized_konsert_datum'] == date]
    rows2 = df2[df2['normalized_konsert_datum'] == date]
    
    # Initialize a list to store comparison results
    comparison_results = []
    
    # Compare each row in rows2 with each row in rows1
    for idx2, row2 in rows2.iterrows():
        best_match_score = 0
        best_match_idx1 = None
        best_match_scores = {}
        for idx1, row1 in rows1.iterrows():
            total_score = 0
            column_scores = {}
            for col in columns:
                score = fuzz.token_sort_ratio(row1[col], row2[col])
                total_score += score
                column_scores[col] = score
            avg_score = total_score / len(columns)
            if avg_score > best_match_score:
                best_match_score = avg_score
                best_match_idx1 = idx1
                best_match_scores = column_scores
        
        # Append results for this row
        comparison_results.append({
            'index': idx2,
            'best_match_index': best_match_idx1,
            'best_match_score': best_match_score,
            'column_scores': best_match_scores
        })
    
    return comparison_results

# Group rows by normalized date and compare
all_comparison_results = []
unique_dates = pd.concat([xls_df['normalized_konsert_datum'], json_df['normalized_konsert_datum']]).unique()

for date in unique_dates:
    if date:  # Ensure date is not empty
        comparison_results = compare_rows_on_date(date, xls_df, json_df, columns_to_compare)
        all_comparison_results.extend(comparison_results)

# Analyze results
for result in all_comparison_results:
    print(f"Row index in LLM data: {result['index']}")
    print(f"Best match row index in human data: {result['best_match_index']}")
    print(f"Best match score: {result['best_match_score']}")
    for col, score in result['column_scores'].items():
        print(f"  {col}: {score}")
    print()

# Convert comparison results to DataFrame for further analysis
comparison_df = pd.DataFrame(all_comparison_results)

# Optionally, save the comparison results to a CSV file
comparison_df.to_csv('detailed_comparison_results.csv', index=False)


Row index in LLM data: 39
Best match row index in human data: 1
Best match score: 60.333333333333336
  konsert_namn: 53
  lokal_namn: 100
  konsert_biljettpris: 55
  Arrangör: 100
  konserttyp_namn: 22
  Producer: 32

Row index in LLM data: 40
Best match row index in human data: 1
Best match score: 75.5
  konsert_namn: 66
  lokal_namn: 100
  konsert_biljettpris: 55
  Arrangör: 100
  konserttyp_namn: 100
  Producer: 32

Row index in LLM data: 41
Best match row index in human data: 9
Best match score: 71.66666666666667
  konsert_namn: 49
  lokal_namn: 100
  konsert_biljettpris: 39
  Arrangör: 75
  konserttyp_namn: 100
  Producer: 67

Row index in LLM data: 21
Best match row index in human data: 16
Best match score: 49.166666666666664
  konsert_namn: 42
  lokal_namn: 29
  konsert_biljettpris: 100
  Arrangör: 36
  konserttyp_namn: 52
  Producer: 36

Row index in LLM data: 22
Best match row index in human data: 16
Best match score: 35.5
  konsert_namn: 52
  lokal_namn: 39
  konsert_biljettp

## Find unique concerts for LLM

In [166]:
# Function to find unique concerts based on dates
def find_unique_dates(df1, df2, column_name='normalized_konsert_datum', threshold=80):
    # Concerts found by LLM but not by human
    unique_to_llm = df2[df2[f'score_{column_name}'] < threshold]
    # Concerts found by human but not matched by LLM
    matched_indices = df2[df2[f'score_{column_name}'] >= threshold][f'best_match_{column_name}'].index
    matched_entries = df1.loc[matched_indices]
    unique_to_human = df1[~df1.index.isin(matched_entries.index)]
    return unique_to_llm, unique_to_human

# Find unique concerts based on dates
unique_llm_dates, unique_human_dates = find_unique_dates(xls_df, json_df)

print(f"\nUnique concerts found by LLM but not by human (total {len(unique_llm_dates)}):")
print(unique_llm_dates[['konsert_datum', 'best_match_normalized_konsert_datum', score_col]])

print(f"\nUnique concerts found by human but not by LLM (total {len(unique_human_dates)}):")
print(unique_human_dates[['konsert_datum']])

# Optionally, save the unique concert results to separate CSV files for further analysis
unique_llm_dates.to_csv('unique_to_llm_konsert_datum.csv', index=False)
unique_human_dates.to_csv('unique_to_human_konsert_datum.csv', index=False)


Unique concerts found by LLM but not by human (total 8):
        konsert_datum best_match_normalized_konsert_datum  score_Producer
3          21.02.1908                            No match             100
6                                                No match              46
9                                                No match              73
25                                               No match              53
36                                               No match             100
43  11.02.08 15.02.08                            No match              50
46                                               No match              48
54                                               No match              58

Unique concerts found by human but not by LLM (total 471):
    konsert_datum
2      1908-01-05
3      1908-01-05
4      1908-01-05
6      1908-01-06
9      1908-01-07
..            ...
509    1908-12-26
510    1908-12-26
511    1908-12-27
512    1908-12-31
513    1908-12-31
